# MLflow Example - Langchain

## Quick example with OpenAI LLM:

This is an example of using MLflow to track experiments and deploy models. We will use the Langchain track the model predictions for a given text.

In [1]:
import mlflow
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

### Prepare model and mlflow tracking

Log everything manually... because you can

In [ ]:
# Setup the mlflow connection

mlflow_uri = "http://192.168.30.21:8080/"
mlflow.set_tracking_uri(mlflow_uri)

# Set the experiment name
mlflow.set_experiment("BrutalExperiment")  # maybe change to your own experiment


In [ ]:
# # Load the .env file with OPENAI_API_KEY if not already set in the environment
# import dotenv
# dotenv.load_dotenv()

# This will enable autologging for all the components in the LLM chain
mlflow.langchain.autolog()


# default is gpt-3.5-turbo, you can change it to gpt-4o for better results
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template("Tell me something about {input} in {output_language}:\n")
chain = prompt | llm

with mlflow.start_run(run_name="test_run"):  # mlflow.end_run() will be called automatically
    # Invoking the chain will cause a trace to be logged
    response = chain.invoke(
        {
            "input": "Cognexa",
            "output_language": "Slovak Language",
        }
    )

    print(response.content)

    mlflow.log_metric(
        "tokens_used",
        response.response_metadata.get("token_usage", {}).get("total_tokens", 0),
    )

    mlflow.log_param(
        "model_used",
        response.response_metadata.get("model_name"),
    )

    # mlflow.log_artifact("data.txt")  # This will append file to the run